In [1]:
from minicons import scorer
import pandas as pd
import json
import numpy as np
import re
import tqdm
import os

/usr/local/lib/python3.10/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
cd ../../../work/sftwang/babyLM_TW_FR/

/work/sftwang/babyLM_TW_FR


In [3]:
ls

Complete_LM_base_creation_from_txt.ipynb      data_benchmark/
Create_txt_files_for_LM_training.ipynb        data_raw_txt/
FineTune_and_Evaluate_on_Benchmarks.ipynb     disfl_1219.csv
FineTune_and_Evaluate_on_Benchmarks_V2.ipynb  disfl_1219_comma.csv
FineTune_and_Evaluate_on_Benchmarks_all.py    disfl_results/
FineTune_and_Evaluate_on_Benchmarks_all.sh    disfl_results_comma/
FineTune_and_Evaluate_on_Benchmarks_en.py     disfl_results_eosbos/
FineTune_and_Evaluate_on_Benchmarks_en.sh     disfl_results_nopause/
LM_base_creation_from_txt.ipynb               disfl_results_strip/
LM_base_creation_from_txt_all.sh              figs/
LM_base_creation_from_txt_en.py               fr_full_conv_spbpe_full.model
LM_base_creation_from_txt_en.sh               fr_full_conv_spbpe_full.vocab
LM_base_creation_from_txt_spxlm_base.py       grouped_en.sh
LM_base_creation_from_txt_spxlm_base_V2.py    models/
README.md                                     results/
analysis_en.ipynb


In [4]:
models = ["./models/fr_conv_spbpe_concat",
         "./models/fr_wiki_spbpe_concat",
         "./models/fr_mixed_spbpe_concat",
         "./models/en_spoken_spbpe_concat",
         "./models/en_wiki_spbpe_concat",
         "./models/en_babylm_spbpe_concat",
         "FacebookAI/xlm-roberta-large",
         "FacebookAI/xlm-roberta-base"]

benchmark_path = './data_benchmark/disfl_linebreak/'
benchmarks = [f for f in os.listdir(benchmark_path) if os.path.isfile(os.path.join(benchmark_path, f)) and 'json' in f]
benchmarks[:5]

['moved_filler_en_1.json',
 'moved_filler_en_0.json',
 'moved_filler_en_2.json',
 'moved_filler_en_3.json',
 'moved_filler_en_4.json']

In [5]:
results_folder = './disfl_results_linebreak/'
if not os.path.exists(results_folder):
    os.makedirs(results_folder)
results = []



for ckpt in tqdm.tqdm(models):
    mlm_model = scorer.MaskedLMScorer(ckpt, 'cuda')
    model_name = re.sub('(\.|/)+', '_', ckpt)
    model_name = re.sub('(_babyLM_TW_FR_models_|_models_|_spbpe_concat|FacebookAI_)', '', model_name)
    
    model_ea = results_folder + '/' + model_name + '_error_analysis/'
    if not os.path.exists(model_ea):
        os.makedirs(model_ea)
    
    for b in benchmarks:
        with open(benchmark_path+b, 'r', encoding = 'utf-8') as f:
            f = f.read().split('\n')
            data = [json.loads(e) for e in f if e.strip()]
            
            term = data[0]['linguistics_term']
            UID  = data[0]['UID']

            scores = []
            for i in range(len(data)):
                pairs = [data[i]['sentence_good'], data[i]['sentence_bad']]
                pairs_scores = mlm_model.conditional_score(['</s>', '</s>'], pairs)
                data[i]['sentence_good_prob'] = pairs_scores[0]
                data[i]['sentence_bad_prob'] = pairs_scores[1]  
                acc = float(pairs_scores[0] > pairs_scores[1])
                data[i]['acc'] = acc
                scores.append(acc)
                
            agg = np.mean(scores)
            with open(model_ea + b, 'w', encoding='utf-8') as outfile:
                for entry in data:
                    json.dump(entry, outfile, ensure_ascii=False)
                    outfile.write('\n')          

            
            out_dict = {'model': model_name, 'file_name': b, 
                        'linguistics_term': term, 'UID': UID, 'score': agg}
            results.append(out_dict)

df = pd.DataFrame(results)
df.set_index('model')
df.to_csv(results_folder + "disfl_1219_linebreak.csv")

 75%|███████▌  | 6/8 [15:23<05:11, 155.82s/it]/home/sftwang/.local/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
100%|██████████| 8/8 [31:02<00:00, 232.76s/it]


In [40]:
results[:5]

[{'model': 'fr_conv',
  'file_name': 'replaced_filler_en',
  'linguistics_term': 'moved_filler_en',
  'UID': 'moved_filler_en_1',
  'score': 0.4602510460251046},
 {'model': 'fr_conv',
  'file_name': 'replaced_filler_en',
  'linguistics_term': 'moved_filler_en',
  'UID': 'moved_filler_en_0',
  'score': 0.5313807531380753},
 {'model': 'fr_conv',
  'file_name': 'replaced_filler_en',
  'linguistics_term': 'moved_filler_en',
  'UID': 'moved_filler_en_2',
  'score': 0.4811715481171548},
 {'model': 'fr_conv',
  'file_name': 'replaced_filler_en',
  'linguistics_term': 'moved_filler_en',
  'UID': 'moved_filler_en_3',
  'score': 0.4811715481171548},
 {'model': 'fr_conv',
  'file_name': 'replaced_filler_en',
  'linguistics_term': 'moved_filler_en',
  'UID': 'moved_filler_en_4',
  'score': 0.5146443514644351}]

In [46]:
df = pd.DataFrame(results)
df.set_index('model')
df.to_csv(results_folder + "disfl_1219_stripped.csv")

In [15]:
print(df.loc[df['benchmark'] == 'moved_filler'].set_index('model'))
print(df.loc[df['benchmark'] == 'removed_filler'].set_index('model'))
print(df.loc[df['benchmark'] == 'replaced_filler'].set_index('model'))
print(df.loc[df['benchmark'] == 'shuffled'].set_index('model'))

print(df.loc[df['benchmark'] == 'moved_filler_en'].set_index('model'))
print(df.loc[df['benchmark'] == 'removed_filler_en'].set_index('model'))
print(df.loc[df['benchmark'] == 'replaced_filler_en'].set_index('model'))
print(df.loc[df['benchmark'] == 'shuffled_en'].set_index('model'))

                      benchmark  score
model                                 
fr_conv            moved_filler  0.810
fr_wiki            moved_filler  0.720
fr_mixed           moved_filler  0.805
en_spoken          moved_filler  0.475
en_wiki            moved_filler  0.480
en_babylm          moved_filler  0.435
xlm-roberta-large  moved_filler  0.675
xlm-roberta-base   moved_filler  0.720
                        benchmark  score
model                                   
fr_conv            removed_filler  0.435
fr_wiki            removed_filler  0.350
fr_mixed           removed_filler  0.565
en_spoken          removed_filler  0.155
en_wiki            removed_filler  0.710
en_babylm          removed_filler  0.425
xlm-roberta-large  removed_filler  0.810
xlm-roberta-base   removed_filler  0.740
                         benchmark  score
model                                    
fr_conv            replaced_filler  0.940
fr_wiki            replaced_filler  0.410
fr_mixed           replaced_fill

In [21]:
print(df.loc[df['benchmark'] == 'moved_filler'].set_index('model'))
print(df.loc[df['benchmark'] == 'removed_filler'].set_index('model'))
print(df.loc[df['benchmark'] == 'shuffled'].set_index('model'))
print(df.loc[df['benchmark'] == 'moved_filler_en'].set_index('model'))
print(df.loc[df['benchmark'] == 'removed_filler_en'].set_index('model'))
print(df.loc[df['benchmark'] == 'shuffled_en'].set_index('model'))


                      benchmark  score
model                                 
fr_conv            moved_filler  0.805
fr_wiki            moved_filler  0.690
fr_mixed           moved_filler  0.840
en_spoken          moved_filler  0.485
en_wiki            moved_filler  0.470
en_babylm          moved_filler  0.490
xlm-roberta-large  moved_filler  0.720
xlm-roberta-base   moved_filler  0.710
                        benchmark  score
model                                   
fr_conv            removed_filler  0.385
fr_wiki            removed_filler  0.010
fr_mixed           removed_filler  0.160
en_spoken          removed_filler  0.020
en_wiki            removed_filler  0.565
en_babylm          removed_filler  0.240
xlm-roberta-large  removed_filler  0.665
xlm-roberta-base   removed_filler  0.680
                  benchmark  score
model                             
fr_conv            shuffled  1.000
fr_wiki            shuffled  0.985
fr_mixed           shuffled  1.000
en_spoken          shuffl

In [7]:
df.loc[df['benchmark'] == 'removed_filler'].set_index('model')

,benchmark,score
model,,
fr_conv,removed_filler,0.435
fr_wiki,removed_filler,0.350
fr_mixed,removed_filler,0.565
en_spoken,removed_filler,0.155
en_wiki,removed_filler,0.710
en_babylm,removed_filler,0.425
xlm-roberta-large,removed_filler,0.810
xlm-roberta-base,removed_filler,0.740


In [8]:
df.loc[df['benchmark'] == 'shuffled'].set_index('model')

,benchmark,score
model,,
fr_conv,shuffled,1.000
fr_wiki,shuffled,0.975
fr_mixed,shuffled,0.995
en_spoken,shuffled,0.645
en_wiki,shuffled,0.695
en_babylm,shuffled,0.620
xlm-roberta-large,shuffled,1.000
xlm-roberta-base,shuffled,1.000


In [9]:
df.loc[df['benchmark'] == 'shuffled_sanity'].set_index('model')

,benchmark,score
model,,
fr_conv,shuffled_sanity,0.370
fr_wiki,shuffled_sanity,0.045
fr_mixed,shuffled_sanity,0.225
en_spoken,shuffled_sanity,0.000
en_wiki,shuffled_sanity,0.065
en_babylm,shuffled_sanity,0.000
xlm-roberta-large,shuffled_sanity,0.950
xlm-roberta-base,shuffled_sanity,0.940


In [10]:
df.loc[df['benchmark'] == 'moved_filler_en'].set_index('model')

,benchmark,score
model,,
fr_conv,moved_filler_en,0.485356
fr_wiki,moved_filler_en,0.539749
fr_mixed,moved_filler_en,0.527197
en_spoken,moved_filler_en,0.715481
en_wiki,moved_filler_en,0.573222
en_babylm,moved_filler_en,0.719665
xlm-roberta-large,moved_filler_en,0.728033
xlm-roberta-base,moved_filler_en,0.728033


In [61]:
ckpt = "../babyLM_TW_FR/models/fr_conv_spbpe_concat"

In [62]:
mlm_model = scorer.MaskedLMScorer(ckpt, 'cuda')
mlm_model.bos_token_id

0

In [63]:
mlm_model.add_special_tokens("ts")

['<s> ts </s>']

In [64]:
benchmark_path = 'evaluation_data/disfl/moved_filler.json'
with open(benchmark_path, 'r', encoding = 'utf-8') as f:
    f = f.read().split('\n')
    data = [json.loads(e) for e in f if e.strip()]

In [65]:
i=4
pairs = [data[i]['sentence_good'], data[i]['sentence_bad']]
pairs

["on en pouvait plus de euh d'attendre *",
 "on en pouvait plus euh de d'attendre *"]

In [66]:
results = mlm_model.conditional_score(['</s>'], [" c' était euh c' était quand je bossais en Belgique euh #"])
print(results)


[-2.4826993703842164]


In [67]:
mlm_model.sequence_score(pairs, reduction = lambda x: x.mean(0).item())
    

[-2.904822667439779, -2.7661989529927573]

In [68]:
mlm_model.conditional_score([" dans le sens où il euh il demande beaucoup ", " gens dans beaucoup aux le il demande euh il"], 
                            ["aux gens", "sens où"])


[-2.879453659057617, -10.80995798110962]

100%|██████████| 8/8 [04:14<00:00, 31.86s/it]


In [ ]:
df.loc[df['benchmark'] == 'removed_filler_en'].set_index('model')

In [ ]:
df.loc[df['benchmark'] == 'shuffled_en'].set_index('model')

In [ ]:
np.mean([x[0] > x[1] for x in results])

In [ ]:
results

In [49]:
import torch
#from torch.nn.parallel import DataParallel as DP
import composer.functional as cf
from composer.algorithms import Alibi
import composer.trainer as ct
from composer.metrics.nlp import LanguageCrossEntropy, MaskedAccuracy, BinaryF1Score
from composer.models.huggingface import HuggingFaceModel
from composer.metrics import CrossEntropy
from torchmetrics.classification import MulticlassAccuracy


import os
import shutil
import json
import numpy as np
import re

####
from pathlib import Path
from datasets import *
from torch.utils.data import DataLoader
from transformers import *
from tokenizers import *
from tokenizers.models import BPE
from tokenizers.trainers import BpeTrainer


from transformers import DataCollatorForLanguageModeling
from transformers import LineByLineTextDataset
from transformers import is_torch_tpu_available

import random
import argparse

/home/sftwang/.local/lib/python3.10/site-packages/transformers/deepspeed.py:24: FutureWarning: transformers.deepspeed module is deprecated and will be removed in a future version. Please import deepspeed modules directly from transformers.integrations
  warnings.warn(
Discovered apex.normalization.FusedRMSNorm - will use it instead of LongT5LayerNorm
Discovered apex.normalization.FusedRMSNorm - will use it instead of Pix2StructLayerNorm
Discovered apex.normalization.FusedRMSNorm - will use it instead of Pop2PianoLayerNorm
Discovered apex.normalization.FusedRMSNorm - will use it instead of T5LayerNorm


In [53]:
tokenizer = AutoTokenizer.from_pretrained("./models/fr_conv_spbpe_concat")

loading file sentencepiece.bpe.model
loading file tokenizer.json
loading file added_tokens.json
loading file special_tokens_map.json
loading file tokenizer_config.json


In [54]:
tokenizer("this is um <unk>").tokens()

['<s>', '▁t', 'hi', 's', '▁', 'is', '▁', 'um', '<unk>', '</s>']

In [94]:
mlm_model = scorer.MaskedLMScorer("FacebookAI/xlm-roberta-large", 'cuda')

loading configuration file config.json from cache at /home/sftwang/.cache/huggingface/hub/models--FacebookAI--xlm-roberta-large/snapshots/c23d21b0620b635a76227c604d44e43a9f0ee389/config.json
Model config XLMRobertaConfig {
  "_name_or_path": "FacebookAI/xlm-roberta-large",
  "architectures": [
    "XLMRobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 1024,
  "initializer_range": 0.02,
  "intermediate_size": 4096,
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "xlm-roberta",
  "num_attention_heads": 16,
  "num_hidden_layers": 24,
  "output_past": true,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "transformers_version": "4.45.1",
  "type_vocab_size": 1,
  "use_cache": true,
  "vocab_size": 250002
}

loading file sentencepiece.bpe.model from cache at /home/sftwang/.cache/huggingface/hu

In [98]:
mlm_model.conditional_score(['</s>', '</s>'], ["je crois j' étais euh je me rappelle plus si j' étais on était au lycée ou en # en première année de fac je crois #",'這根本是 中文'])

[-2.119367480278015, -4.708889007568359]